In [ ]:
import subprocess
import os
import shutil
import numpy as np
import pandas as pd

In [ ]:
path="/home/cptd/Documents/Molclus/fangbo"
molculs_path = f"/home/cptd/Documents/Molclus/molclus_1.12_Linux"
new_file_name = 'traj.xyz'
destination_folder = 'genmer_mol'
destination_file_path=os.path.join(path, destination_folder)

In [ ]:
def get_file_names_in_directory(path):
    files = []
    file_names =[]
    files_names = []
    for file in os.listdir(path):
        files_names.append(file)
        file_name = file.split(".")[0]
        file_names.append(file_name)
        if os.path.isfile(os.path.join(path, file)):
            files.append(os.path.join(path, file))
    return files,file_names,files_names

In [ ]:
def get_atom_num(genmer_files_names,i):
    mol1 = genmer_files_names[i].split("_")[0]
    mol2 = genmer_files_names[i].split("_")[1].split(".")[0]
    atom1 = df.loc[df['names'] == mol1, 'atom_num'].values[0]
    atom2 = df.loc[df['names'] == mol2, 'atom_num'].values[0]
    line1 = f"1-{atom1}"
    line2 = f"{atom1+1}-{atom1+atom2}"
    atomnum=atom1+atom2
    return line1,line2,atomnum

In [ ]:
def Ti_file_xyz(input_file,output_file,atomnum): 
    with open(input_file, 'r') as file:
        lines = file.readlines()
    header_lines = lines[:atomnum+2]
    with open(output_file, 'w') as new_file:
        new_file.writelines(header_lines)   

In [ ]:
def shell_change(line1,line2,i):
# Read raw shell script file
    with open('multiwfn_bind.sh', 'r') as file:
        lines = file.readlines()
# Amendments to lines 1, 9 and 10
    lines[0] = f'Multiwfn {genmer_names[i]}_best1.xyz > /dev/null << EOF\n'
    lines[8] = f'{line1}\n'
    lines[9] = f'{line2}\n'
# Save the modifications as a new shell script file
    with open('modified_multiwfn.sh', 'w') as new_file:
        new_file.writelines(lines)
    print("The new Shell script file is successfully modified and saved")    

In [ ]:
genmer_file_paths,genmer_names,genmer_files_names = get_file_names_in_directory(destination_file_path)

In [ ]:
df = pd.read_excel("atom_num_name.xlsx")

In [ ]:
from tqdm import tqdm

In [ ]:
#Create a new file path
new_folder_path = f'{molculs_path}/401'
os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
for i in tqdm(range(len(genmer_files_names))):
    line1,line2,atomnum=get_atom_num(genmer_files_names,i)
    destination_file = os.path.join(molculs_path, new_file_name)
    shutil.copy(genmer_file_paths[i], destination_file)
    os.rename(destination_file, os.path.join(molculs_path, new_file_name))     
    os.chdir(molculs_path)
    os.system("bash molclus.sh")
    # 读取原始XYZ文件
    input_file='cluster.xyz'
    output_file=f'{genmer_names[i]}_best1.xyz'
    Ti_file_xyz(input_file,output_file,atomnum)
    shell_change(line1,line2,i)
    os.system("bash modified_multiwfn.sh")
    sub_folder_path = os.path.join(new_folder_path, f'{genmer_names[i]}')
    try:
        os.makedirs(sub_folder_path,exist_ok=True)
    except:
        pass
    #SP_path = os.path.join(sub_folder_path,f"{genmer_names[i]}")
    shutil.move("SP.inp", sub_folder_path)

In [ ]:
new_folder_path = f'{molculs_path}/401'
root_folder = new_folder_path
SP_paths=[]
Out_paths=[]
gbw_paths=[]
for root, dirs, files in os.walk(root_folder):
    for file in files:
        file_out_path=os.path.join(root, "SP.out")
        gbw_paths.append(os.path.join(root, "SP.gbw"))
        file_path = os.path.join(root, file)
        SP_paths.append(file_path)
        Out_paths.append(file_out_path)

In [ ]:
error_SP_path=[]
error_OUT_path=[]
num=1
for i in tqdm(range(len(SP_paths))):
    SP_file = SP_paths[i]
    Out_file = Out_paths[i]
    if "moa1_" in SP_file:
        error_SP_path.append(SP_file)
        error_OUT_path.append(Out_file)
        #continue
    #if "mob1" in SP_file:
        #continue
    else:
        num+=1
        os.system(f"/home/cptd/Software/ORCA-5.0.3/orca {SP_file} > {Out_file}")
print(num)

In [ ]:
Out_paths=list(set(Out_paths))
len(Out_paths)

In [ ]:
def bind_cal(SP_path_data):
    with open(SP_path_data,"r") as file:
        bind_data = []
        for line in file:
            line = line.strip()
            if "FINAL SINGLE POINT ENERGY" in line:
                bind_data.append(line.split(" ")[-1])
        if len(bind_data)==3:
            #print(SP_path_data)
            total_energy=float(bind_data[0])
            try:
                bind_energy=627.51*(float(bind_data[0])-float(bind_data[1])-float(bind_data[2]))
                result=bind_energy
            except:
                print(SP_path_data)
                result=0
        else:
            result=0
            total_energy=float(bind_data[0])
            #print(genmer_name)
    return result, total_energy

In [ ]:
none_OUT_path = []
none_SP_path = []
none_genmer_name = []
ok_genmer_name = []
total_energys=[]
binds = []
for i in range(len(Out_paths)):
    Out_file = Out_paths[i]
    bind,total_energy = bind_cal(Out_file)
    total_energys.append(total_energy)
    if bind==0:
        none_OUT_path.append(Out_file)
        none_SP_path.append(SP_paths[i])
        none_genmer_name.append(genmer_names[i])
    else:
        binds.append(bind)
        ok_genmer_name.append(genmer_names[i])
    #except:
           # none_OUT_path.append(Out_file)
            #none_SP_path.append(SP_paths[i])
            #none_genmer_name.append(genmer_names[i])

In [ ]:
bind_df_total = pd.DataFrame(np.array([genmer_names,total_energys]).T,columns=["genmer_names","total_energys"])
bind_df_total.to_csv(f"{path}/401_binds_total.csv",index=None)

In [ ]:
bind_df = pd.DataFrame(np.array([ok_genmer_name,binds]).T,columns=["genmer_names","Bind(kcal/mol)"])

In [ ]:
bind_df.to_csv(f"{path}/401_binds.csv",index=None)

In [ ]:
#Separate saving of components that did not run successfully
none_bind_df = pd.DataFrame(np.array(none_genmer_name),columns=["none_genmer_names"])
none_bind_df.to_csv(f"{path}/none.csv",index=None)

# IGMH analysis

In [ ]:
gbw_paths=list(set(gbw_paths))

In [ ]:
#These allow you to select the components you want to calculate and analyze
a1=["moa1","moa2","moa3","moa4","moa5"]
b1=["mob1","mob2","mob3","mob4","mob5"]

In [ ]:
ab=[]
for i in a1:
    for j in b1:
        ab.append(f"{i}_{j}")

In [ ]:
gbw_paths_ok=[]
for i in gbw_paths:
    gbw_name = i.split("/")[-2]
    if gbw_name in ab:
        gbw_paths_ok.append(i)

In [ ]:
genmer_files_names_ok=[]
for i in genmer_files_names:
    gbw_name = i.split(".")[0]
    if gbw_name in ab:
        genmer_files_names_ok.append(i)

In [ ]:
gbw_paths_ok=sorted(gbw_paths_ok)

In [ ]:
def IGMH_analysis(line1,line2,i):
# 读取原始Shell脚本文件
    with open('IGMH.sh', 'r') as file:
        lines = file.readlines()
# 修改第1行，第9行和第10行
        lines[0] = f'Multiwfn {gbw_paths_ok[i]} > /dev/null << EOF\n'
        lines[4] = f'{line1}\n'
        lines[5] = f'{line2}\n'
# 将修改后的内容保存为新的Shell脚本文件
    with open('IGMH_change.sh', 'w') as new_file:
        new_file.writelines(lines)
    print("Successfully modified and saved the new IGMH Shell script file")  

In [ ]:
genmer_files_names=genmer_files_names_ok

In [ ]:
line1,line2,atomnum=get_atom_num(genmer_files_names,1)

In [ ]:
genmer_files_names[1],line1,line2,atomnum

In [ ]:
for i in tqdm(range(len(genmer_files_names))):
    os.chdir(molculs_path)
    line1,line2,atomnum=get_atom_num(genmer_files_names,i)
    IGMH_analysis(line1,line2,i)
    source_file = "IGMH_change.sh"
    sub_folder_path = os.path.join(new_folder_path, f'{ab[i]}')
    
    target_file = os.path.join(sub_folder_path,os.path.basename(source_file))
    if os.path.exists(target_file):
        os.remove(target_file)
        
    shutil.move(source_file, sub_folder_path)
    os.chdir(sub_folder_path)
    os.system(f"bash {sub_folder_path}/IGMH_change.sh")